In [1]:

from bs4 import BeautifulSoup                        # Importamos librerías para extracción y manipulación de datos 
import requests
import pandas as pd
import re

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.
pd.set_option('display.width', 1000)        # Ajusta el ancho de la consola para evitar truncamientos


# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

In [3]:

def llamar_api (year, genero):                       
    
    # Este método intenta llamar a la API Cinema Movies. 
    # Recibe como parámetro el género y el año en que desea buscar y devuelve una lista de tuplas con los datos de cada película.

    #Construimos la URL dinámica con los parámetros recibidos:

    url = f"https://dev.adalab.es/api/cinema/movies?year={year}&genre={genero}" 

    lista_peliculas = []

    respuesta = requests.get(url)

    sleep(3)

    if respuesta.status_code != 200:
        print(f"Error al haber la llamada. Motivo: {respuesta.reason}")
        
    else: 
        json_genero_year= respuesta.json()

        # Recorro el diccionario que me devuelve la API hasta llegar al diccionario que almacena 
        # los títulos, ids, tipos y años de estreno. Lo itero y extraigo los datos de cada película. 
            
        for pelicula in json_genero_year["results"]:   
                                       
            type = pelicula["type"]

            if type == "Movie":     # Filtramos por película. Ignoramos series y otros formatos de vídeo.

                title = pelicula["title"]
                year = pelicula["year"]
                genero = pelicula ["genre"]
                id = pelicula["id"]   
                idOwner = pelicula["idOwner"]

                datos_pelicula = (type, title, year, genero, id, idOwner)           # En cada iteración, creo una tupla con los datos que añado a mi lista. 
                lista_peliculas.append(datos_pelicula)
            
            else:
                pass        

        return lista_peliculas 
    


In [4]:

def todas_las_peliculas_del_anyo (year) :

    # Devuelve las películas de todos los géneros del año pasado por parámetro.
    # Devuelve una lista de tuplas con los datos de cada película.
    # Además, nos será útil para obtener los identificadores de las películas.

    generos = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]
    lista_pelis_total= []

    for genero in generos: 
        lista_pelis_genero = llamar_api(year, genero) 
        lista_pelis_total.extend(lista_pelis_genero)

    return lista_pelis_total

In [5]:
def extraer_identificadores (lista):
    
    # Devuelve una lista con los identificadores de las películas que recibe por parámetro.

    lista_identificadores = []

    for tupla in lista:
        lista_identificadores.append(tupla[-1]) # En la última posición almacenamos el identificador idOwner.

    return lista_identificadores

In [6]:
def escribir_fichero (df, year_inicial, periodo):

    # Recibe un dataframe, lo pasa a .csv y personaliza el nombre del fichero en función del periodo abarcado.

    if periodo == 1:
        nombre_archivo = f'peliculas_{year_inicial}.csv'
        # df.to_csv(f'peliculas_{year_inicial}.csv') 
        df.to_csv(nombre_archivo) 

    else:
        nombre_archivo = f'peliculas_{year_inicial}_{year_inicial+periodo-1}.csv'
        # df.to_csv(f'peliculas_{year_inicial}_{year_inicial+periodo-1}.csv')   
        df.to_csv(nombre_archivo)           

    print(f"Extracción completa. Puedes consultar tu archivo {nombre_archivo}")

In [7]:

def extraer_datos_peliculas_periodo (year_inicial, periodo = 1):   
    
    # Devuelve un dataframe con los datos de todas las películas del periodo pasado por parámetro.
    # Escribe los datos del dataframe en un fichero. csv con título personalizado.
    # Se recomienda que periodo no exceda de 5 años.

    lista_periodo = []

    if (periodo > 0) and (year_inicial >= 1900):                  

        # Iteramos en los años del periodo marcado y extraemos los datos de todas las películas de ese año. 

        for year in range (year_inicial, year_inicial+periodo):
            try:
                print(f"Explorando películas del año {year}")
                lista_pelis_anual = todas_las_peliculas_del_anyo(year)
                lista_periodo.extend(lista_pelis_anual)
            except:
                print(f"No ha sido posible obtener los datos. Recomendamos introducir periodos máximos de 5 años")
                return

        encabezados = ["Tipo", "Título", "Año", "Género", "Id_API", "IdOwner"]
        df = pd.DataFrame(lista_periodo, columns = encabezados) 
        escribir_fichero (df, year_inicial, periodo) # Escribimos los datos en fichero .csv
        return df
    
    else:
        print("Los parámetros introducidos no son válidos. \nPor favor, revise el año inicial o el periodo")  


Probamos a extraer películas de dos periodos.

- El año 1984.

- El periodo 2000-2004. 


In [8]:
df_pelis_1984 = extraer_datos_peliculas_periodo(1984)
df_pelis_1984.head(10)

Explorando películas del año 1984
Extracción completa. Puedes consultar tu archivo peliculas_1984.csv


,Tipo,Título,Año,Género,Id_API,IdOwner
0,Movie,Bástyasétány hetvennégy,1984,Drama,51564,tt0071261
1,Movie,Cruz de olvido,1984,Drama,51566,tt0080570
2,Movie,Qing ren kan dao,1984,Drama,51567,tt0081716
3,Movie,Rare Breed,1984,Drama,51568,tt0082972
4,Movie,Die Frau mit dem roten Hut,1984,Drama,51569,tt0083969
5,Movie,Naitou l'orpheline,1984,Drama,51570,tt0084384
6,Movie,Napló gyermekeimnek,1984,Drama,51571,tt0084388
7,Movie,Habanera,1984,Drama,51572,tt0082491
8,Movie,The Dream Chasers,1984,Drama,51573,tt0083852
9,Movie,La gran quiniela,1984,Comedy,51608,tt0082469


In [10]:
df_pelis_2000_2004 = extraer_datos_peliculas_periodo(2000, 5)
df_pelis_2000_2004.head(25)


Explorando películas del año 2000
Explorando películas del año 2001
Explorando películas del año 2002
Explorando películas del año 2003
Explorando películas del año 2004
Extracción completa. Puedes consultar tu archivo peliculas_2000_2004.csv


,Tipo,Título,Año,Género,Id_API,IdOwner
0,Movie,Istota,2000,Drama,56524,tt0102362
1,Movie,For the Cause,2000,Drama,56526,tt0113092
2,Movie,Florentino y el diablo,2000,Drama,56527,tt0113086
3,Movie,Karobaar: The Business of Love,2000,Drama,56528,tt0116748
4,Movie,Gang,2000,Drama,56530,tt0116391
5,Movie,Consequence,2000,Drama,56531,tt0115937
6,Movie,Fa yeung nin wah,2000,Drama,56532,tt0118694
7,Movie,Love 101,2000,Comedy,56567,tt0119495
8,Movie,Nothing Sacred,2000,Comedy,56568,tt0119806
9,Movie,State and Main,2000,Comedy,56569,tt0120202
